In [87]:
%pip install gspread
%pip install contractions


  Using cached contractions-0.1.73-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached textsearch-0.0.24-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached anyascii-0.3.2-py3-none-any.whl.metadata (1.5 kB)
  Using cached pyahocorasick-2.1.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
Using cached contractions-0.1.73-py2.py3-none-any.whl (8.7 kB)
Using cached textsearch-0.0.24-py2.py3-none-any.whl (7.6 kB)
Using cached anyascii-0.3.2-py3-none-any.whl (289 kB)
Using cached pyahocorasick-2.1.0-cp312-cp312-win_amd64.whl (39 kB)
Note: you may need to restart the kernel to use updated packages.


In [92]:
import praw, random, pandas as pd, time
from google.cloud import bigquery
import contractions
import datetime
import random, os, json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import gspread
from google.oauth2.service_account import Credentials
import csv
import re

Google Sheet API Connect

In [93]:
scopes = ["https://www.googleapis.com/auth/spreadsheets"]

creds = Credentials.from_service_account_file('google_sheets_api.json', scopes=scopes)

client = gspread.authorize(creds)

sheet_id ="1ZDDw04qT0TqZ4LrwZzEJD5WEphOF6t9B6IU3HDWDRpU"
sheet = client.open_by_key(sheet_id)

In [94]:

general_hiring = sheet.get_worksheet(0)  # Get the first worksheet
values_list = general_hiring.col_values(1)  # Get all values from the first column

print(values_list)

['2D', '3D', 'animation', 'application', 'applications', 'aptitude', 'art', 'assessment', 'asset', 'audience', 'awards', 'background', 'building', 'campaign', 'candidate', 'candidates', 'career', 'challenges', 'collaboration', 'collaborative', 'communication', 'community', 'concept design', 'demo', 'design', 'development', 'discord', 'engagement', 'environment', 'equity', 'events', 'experience', 'feedback', 'freelance', 'full-time', 'github', 'gold', 'growth', 'hiring', 'hunter', 'hybrid', 'industry', 'modeling', 'motivation', 'negotiation', 'network', 'networking', 'office', 'onboarding', 'online', 'opportunities', 'opportunity', 'organization', 'orientation', 'part-time', 'passion', 'performance', 'performance', 'pixel', 'policies', 'portfolio', 'presentation', 'process', 'professional', 'professional', 'programmer', 'programming', 'promotion', 'raise', 'recruitment', 'reddit', 'release', 'remote', 'reporting', 'requirements', 'research', 'resolution', 'resume', 'review', 'review', '

In [95]:
game_dev_words = [
    'adventure', 'adventurer', 'alchemy', 'archmage', 'assassin', 'avatar', 'barbarian', 'bard', 
    'battlefield', 'berserker', 'campaign', 'cleric', 'companion', 'covenant', 'crusade', 
    'dungeon', 'enchantment', 'epic', 'fey', 'fiend', 'gnoll', 'gnome', 'goblin', 
    'golem', 'guild', 'halfling', 'mage', 'magical', 'medieval', 'mercenary', 'mystic', 'myth', 
    'necromancer', 'paladin', 'phoenix', 'quest', 'ranger', 'rogue', 'sorcerer', 'summoner', 
    'sword', 'tavern', 'tome', 'treasure', 'tribe', 'troll', 'unicorn', 'vampire', 'warrior', 
    'wizard', 'wyrm', 'wyvern', 'XP', 'Promotion', 'Marketing', 'Trailer', 'Early Access', 'Beta', 
    'Alpha', 'Launch', 'Steam', 'Console', 'PC', 'Mobile', 'VR', 'AR', 'Gaming', 'Play', 'Modding', 
    'Mods', 'Updates', 'DLC', 'Expansion', 'Multiplayer', 'Singleplayer', 'Co-op', 'Crossplay', 
    'Networking', 'Matchmaking', 'Leaderboards', 'Microtransactions', 'Esports', 'Streamer', 
    'Content Creator', 'YouTuber', 'Twitch', 'Mixer', 'Discord', 'Community Manager', 'Fan', 
    'Fan Art','Forum', 'Blog', 'Social Media', 
    'Reddit', 'Subreddit', 'Post', 'Comment','Share', 'Subscribe', 'Follow', 'Hype', 'Launch', 
    'Release ', 'release Date', 'Launch Trailer', 'Teaser', 'Pre-order', 'Exclusive', 'Early Bird', 
    'Game of the Year', 'Award-winning', 'Top-rated', 'Critical Acclaim', 'Fan', 'Remastered', 'Edition', 'Game Pass', 
    'Season Pass', 'Expansion Pass', 'Downloadable Content', 'Bundle Deal', 'Early Access Game', 'Game','Preview', 
    'Pre-order Bonus', 'Launch', 'Event', 'Community Event', 'Tournament', 'Developer Livestream', 'Beta Test'
]

hire_game_dev_words = ["2D","3D","animation", "application", "applications", "aptitude", "art", 
                  "assessment", "asset", "audience", "awards", "background", "building", "campaign", "candidate", "candidates", 
"career", "challenges", "collaboration", "collaborative", "communication", "community", "concept design"
 "cv", "demo", "design", "development", "gold", "treasure", "hunter"
"digital", "discord", "environment", "engagement", "equity","events", "experience", "feedback", 
"freelance", "full-time", "github", "growth", "hiring", "hybrid", "industry", 
 "modeling", "motivation", "network", "networking", "negotiation", "office", "onboarding", "online", 
"opportunities", "opportunity", "organization", "orientation", "part-time", "passion", "performance", 
"performance", "pixel", "policies", "portfolio", "presentation", "process", "professional", 
"professional", "programmer", "programming", "promotion", "raise", "recruitment", "reddit", "release", 
 "remote", "reporting", "requirements", "research", "resolution", "resume", "review", 
"review", "scrum","share", "slack", "skills", "social",  "strategy", "studio",
  "team","teamwork", "trailer", "training","unity", "unreal", "vacancy","XP"]

vfx_related_words = [
    "3D rendering", "acceleration", "adaptive exposure", "adaptive tessellation", "ambient occlusion",
    "anisotropic filtering", "anti-aliasing", "architectural visualization", "artificial intelligence",
    "atmospheric scattering", "atmospheric effects", "aurora", "billboarding", "binary space partitioning",
    "bloom", "bokeh", "bump mapping", "camera mapping", "cartoon rendering", "caustics", "cel shading",
    "chromatic aberration", "cloth simulation", "collision detection", "color correction", "color grading",
    "composite mapping", "computer-generated imagery", "CPU rendering", "culling", "depth of field", "diffusion",
    "displacement mapping", "dissolve", "distance fog", "dithering", "dynamic lighting", "dynamic shadows",
    "edge detection", "environment mapping", "explosion effects", "fake reflection", "feedback", "fire effects",
    "fish-eye lens", "fluid simulation", "fog", "fractal", "frame rate", "gamma correction", "glare",
    "global illumination", "glow", "god rays", "GPU rendering", "grain", "graphics processing unit",
    "HDR lighting", "height mapping", "image-based lighting", "impostor", "infinite plane", "inverse kinematics",
    "jitter", "kinematic", "lens flare", "light bloom", "light map", "light probes", "lighting",
    "liquid simulation", "local illumination", "LOD (Level of Detail)", "look-up table", "marching cubes",
    "matte painting", "mesh", "metaballs", "micro-polygon rendering", "misfire", "modeling", "motion blur",
    "multi-pass rendering", "noise", "normal mapping", "occlusion culling", "offline rendering", "onion skinning",
    "opacity mapping", "optimization", "orthographic projection", "out-of-core rendering", "particle effects",
    "pathfinding", "perlin noise", "phong shading", "physically-based rendering", "pixel art", "pixelation",
    "plasma effect", "point cloud", "point light", "post-processing", "pre-computed radiance transfer",
    "procedural generation", "projective texturing", "pyrotechnics", "radiosity", "ray casting", "ray tracing",
    "reflection mapping", "refraction", "render farm", "rendering equation", "resampling", "retopology", "rigging",
    "rim lighting", "roto-scope", "sandbox", "scene graph", "screen space", "shader", "shadow mapping", "shape key",
    "skeletal animation", "skin shading", "skybox", "soft body dynamics", "soft shadows", "solid texturing",
    "specular highlights", "sprite", "SSAA (Super-Sampling Anti-Aliasing)", "stencil buffer", "subdivision surface",
    "subsurface scattering", "tangent space", "terrain generation", "texture", "texture filtering", "texture mapping",
    "thermography", "three-dimensional graphics", "time-lapse", "toon shading", "transparency", "translucency",
    "transparency mapping", "turbulence", "UV mapping", "vaporization", "vignetting", "volumetric lighting", "voxel",
    "water simulation", "wavefront", "wireframe", "xenomorphing", "z-buffering", "z-fighting", "zoom blur"
]

digital_art_related_words = [
    "abstract", "acrylics", "airbrush", "ambient occlusion", "analog", "animation", "art", "artist", "artwork",
    "background", "balance", "brush", "canvas", "cartoon", "character", "color", "composition", "concept",
    "concept art", "contrast", "corel painter", "creativity", "cubism", "custom brushes", "design", "digital",
    "digital painting", "doodle", "drawing", "dynamic symmetry", "environment", "expressionism", "fantasy",
    "figure", "figure drawing", "fine art", "flat design", "form", "gesture", "gimp", "graphic design",
    "grayscale", "hatching", "illustration", "impressionism", "ink", "inspiration", "krita", "landscape",
    "layer", "lighting", "line", "linocut", "manga", "masking", "mixed media", "modeling", "monochromatic",
    "motion graphics", "natural media", "oil painting", "opacity", "outline", "palette", "pencil", "perspective",
    "photo manipulation", "photoshop", "pixel art", "pop art", "portrait", "pose", "poster", "procreate",
    "realism", "rendering", "retouching", "scale", "scene", "sculpture", "seascape", "shade", "shadow",
    "shape", "silhouette", "sketch", "software", "space", "speed painting", "spray paint", "still life",
    "storyboard", "style", "stylus", "symmetry", "texture", "thumbnail", "tone", "traditional", "transparency",
    "vector", "watercolor", "web design", "wildlife", "abstract expressionism", "adobe illustrator",
    "adobe photoshop", "aesthetics", "anatomy", "animation software", "architectural visualization", "art book",
    "art deco", "art history", "art nouveau", "artistic style", "assemblage", "avatar", "background design",
    "ballpoint pen", "bargue drawing", "baroque", "bezier curve", "biomorphic", "black and white",
    "blocking in", "blueprint", "branding", "bristle brush", "canvas texture", "cartooning", "cel animation",
    "chalk", "charcoal", "clip studio paint", "collage", "color harmony", "color palette", "color theory",
    "comic book", "conceptual art", "contrast ratio", "copic markers", "corel draw", "craft", "craftsmanship",
    "creative process", "cubist", "cultural context", "curvature", "cyanotype", "decoupage", "digital art",
    "digital brush", "digital drawing", "digital illustration", "digital imaging", "digital medium",
    "digital sketching", "doodle art", "drafting", "drawing tablet", "drip painting", "dutch angle",
    "dynamic composition", "earth colors", "eccentric", "educational", "emotional expression", "encaustic",
    "engraving", "environment design", "expressionist", "fashion illustration", "felt-tip pen", "figure study",
    "floral design", "foreground", "frame", "fresco", "futurism", "gesture drawing", "glitch art", "gouache",
    "graphic tablet", "grayscale painting", "grotesque", "gustave dore", "hand lettering", "hand-drawn",
    "handmade", "hard edge", "historical context", "history of art", "hue", "hyperrealism", "iconography",
    "illustration software", "illustrator", "impressionist", "ink wash", "inking", "intaglio", "intensity",
    "interior design", "isometric drawing", "italic", "japanese brush", "japanese painting", "kawaii",
    "kinetic art", "laser cut", "lens flare", "light source", "line art", "lithograph", "low-poly",
    "manual dexterity", "marbling", "masterpiece", "matisse", "medieval art", "meditative art", "mixed media art",
    "monochrome", "monoprint", "mural", "narrative art", "naturalistic", "neo-expressionism", "neoclassical",
    "net art", "new media art", "oil paint", "organic shape", "orthogonal lines", "outline drawing",
    "paintbrush", "painting software", "panorama", "papercraft", "papyrus", "pastel", "patina", "pen and ink",
    "pen tablet", "perspective drawing", "pixel", "pointillism", "polychrome", "portrait drawing",
    "poster design", "posterization", "poster paint", "printmaking", "projection mapping", "psychedelic art",
    "public art", "quick sketch", "quill pen", "rainbow palette", "realistic drawing", "rebranding", "repetition",
    "retro style", "rhinoceros", "rock painting", "rough sketch", "ruler", "saturation", "sci-fi art",
    "sculptural", "seamless pattern", "sepia", "serif", "shadowing", "shape design", "sketchbook", "sketching",
    "soft pastels", "space art", "spiral", "stained glass", "stencil", "still life drawing", "storytelling",
    "street art", "stylized", "subtle", "surrealist", "symbolism", "symmetrical balance", "tagging",
    "tattoo art", "technical drawing", "technical pen", "technique", "texturing", "the golden ratio",
    "theme art", "thumbnail sketch", "tint", "tonal value", "toned paper", "tooling leather", "trademark",
    "traditional art", "tracing paper", "tracing", "transparent", "trick art", "triple axel", "triumphal arch",
    "trompe l'oeil", "typography", "underpainting", "vector graphic", "virtual art", "visual art", "vivid",
    "watercolor paper", "watercolor pencil", "watercolor wash", "wave pattern", "webcomic", "wireframe",
    "woodcut", "work in progress", "zen art", "zen doodle"
]

marketing_related_words = [
    "advertising", "branding", "campaign", "digital", "email", "engagement", "funnel", "growth", "influencer",
    "jingle", "keyword", "lead", "market", "networking", "objective", "positioning", "qualified", "reputation",
    "segmentation", "targeting", "user-generated", "viral", "word-of-mouth", "analytics", "buyer", "content",
    "conversion", "customer", "data-driven", "ecommerce", "feedback", "global", "hubspot", "insights", "journey",
    "kpi", "loyalty", "metrics", "nurturing", "optimization", "personalization", "quality", "retargeting",
    "strategy", "tactics", "unique", "value", "webinar", "a/b testing", "b2b", "call-to-action", "demographics",
    "engagement rate", "fomo", "google ads", "heatmap", "inbound", "keywords", "landing page", "marketing mix",
    "native advertising", "omnichannel", "pinterest", "qualified leads", "relevance", "seo", "social media",
    "thought leadership", "usability", "value proposition", "web design", "abm", "b2c", "case study",
    "content marketing", "conversation rate", "customer acquisition", "demand generation", "engagement rate",
    "frequency", "growth hacking", "influencer marketing", "journey mapping", "lead generation", "market research",
    "niche", "optimization", "persona", "qualified leads", "retention rate", "sales funnel", "segment",
    "target audience", "user experience", "viral marketing", "web analytics", "affiliate marketing", "blogging",
    "click-through rate", "conversion rate", "customer journey", "direct marketing", "email marketing",
    "funnel marketing", "guerilla marketing", "influencer outreach", "journey mapping", "keyword research",
    "lead nurturing", "market segmentation", "online marketing", "personal branding", "qualified traffic",
    "remarketing", "search engine marketing", "social media advertising", "thought leadership", "user engagement",
    "viral content", "web presence", "account-based marketing", "blog post", "content creation", "customer data",
    "digital advertising", "engagement strategy", "funnel optimization", "growth strategy", "inbound marketing",
    "keyword optimization", "landing page optimization", "marketing automation", "networking events", "organic traffic",
    "persona development", "quality score", "reputation management", "seo optimization", "social proof",
    "user acquisition", "viral campaign", "web traffic", "ad copy", "b2b marketing", "call-to-action button",
    "content strategy", "conversion funnel", "customer engagement", "demand generation", "engagement rate",
    "google analytics", "heatmap analysis", "influencer outreach", "journey mapping", "keyword targeting",
    "lead generation", "market analysis", "niche market", "online presence", "personalized marketing",
    "qualified leads", "retargeting ads", "sales pipeline", "segmentation strategy", "target market",
    "user behavior", "viral marketing campaign", "website optimization", "affiliate program", "blog content",
    "click-through optimization", "conversion optimization", "customer journey mapping", "direct mail marketing",
    "email automation", "funnel strategy", "growth hacking tactics", "inbound content", "keyword ranking",
    "lead nurturing campaign", "market positioning", "online advertising", "personalization strategy",
    "qualified prospects", "remarketing campaign", "search engine optimization", "social media engagement",
    "thought leadership content", "user-generated content", "viral content marketing", "web presence optimization",
    "account-based marketing strategy", "blogging platform", "content creation process", "customer data analysis",
    "digital marketing", "engagement tactics", "funnel optimization strategy", "growth strategy development",
    "inbound lead generation", "keyword research tool", "landing page design", "marketing automation platform",
    "networking opportunities", "organic search traffic", "persona development process", "quality score improvement",
    "reputation management strategy", "seo strategy", "social proofing", "user acquisition strategy",
    "viral campaign strategy", "web traffic generation", "ad copy optimization", "b2b marketing strategy",
    "call-to-action optimization", "content strategy development", "conversion funnel optimization",
    "customer engagement strategy", "demand generation campaign", "engagement rate improvement",
    "google analytics analysis", "heatmap analysis tool", "influencer outreach strategy", "journey mapping process",
    "keyword targeting strategy", "lead generation campaign", "market analysis process", "niche market identification",
    "online presence"]

modeling_and_sculpting_related_words = [
    "3D", "animation", "art", "Blender", "CAD", "character", "computer-generated", "design", "digital", "graphics",
    "illustration", "lighting", "Maya", "model", "polygon", "rendering", "rigging", "simulation", "sketch", "texture",
    "UV", "wireframe", "ZBrush", "additive", "algorithm", "animation", "architectural", "autodesk", "bump", "Cinema 4D",
    "clay", "concept", "creative", "deformation", "detailing", "digital", "displacement", "drafting", "drawing",
    "environment", "game", "geometry", "GIMP", "graphics", "hard-surface", "inorganic", "landscape", "low poly",
    "material", "mesh", "modeling", "motion", "organic", "photorealistic", "Photoshop", "physically-based", "pixel",
    "procedural", "realistic", "retopology", "sculpting", "shading", "shader", "shape", "simulation", "sketch",
    "soft-body", "subdivision", "subsurface", "surface", "terrain", "texturing", "topology", "vector", "visual",
    "visual effects", "wireframe", "animation", "artistic", "blender", "camera", "cavity", "character", "clay",
    "compositing", "concept", "creature", "detail", "digital", "environment", "facial", "figure", "game", "geometry",
    "hard surface", "high poly", "human", "illustration", "landscape", "light", "low poly", "material", "mesh",
    "modeling", "motion", "normal", "organic", "painting", "photorealistic", "pixel", "polygon", "pose", "rigging",
    "sculpting", "shadow", "shape", "simulation", "softbody", "specular", "texturing", "topology", "UV", "vector",
    "virtual", "visualization", "wire", "zbrush", "3D", "anatomy", "art", "background", "Blender", "canvas", "CGI",
    "concept", "creature", "design", "detail", "digital", "dynamic", "effect", "environment", "fantasy", "figure",
    "frame", "game", "geometry", "graphic", "human", "illusion", "landscape", "lighting", "low-poly", "material",
    "medium", "mesh", "model", "motion", "object", "organic", "painting", "photo", "pixel", "polygon", "portrait",
    "process", "realism", "render", "scene", "sculpt", "shape", "simulation", "software", "structure", "style",
    "surface", "technique", "texture", "tool", "traditional", "transformation", "visualization", "volumetric",
    "abstract", "animate", "architectural", "artwork", "background", "blending", "camera", "carve", "CG", "collage",
    "conceptual", "creature", "design", "detailing", "digital", "draft", "effect", "environmental", "fantasy",
    "film", "geometry", "graphics", "humanoid", "illustration", "landscaping", "light", "low-polygon", "material",
    "medium", "model", "movement", "naturalistic", "object", "organic", "paint", "photo-realistic", "pixelated",
    "polygons", "portrait", "process", "realistic", "rendering", "scene", "sculpture", "shape", "simulation",
    "sketching", "software", "specular", "structure", "stylized", "surface", "technological", "textured", "tools",
    "traditional", "transform", "visual", "volumetric", "animate", "architect", "artistic", "backgrounds", "blend",
    "camera angles", "carving", "cgi", "collage", "conceptualization", "creature design", "details", "digital art",
    "drafting", "effects", "environments", "fantastical", "filmmaking", "geometry", "graphic design", "human figure",
    "illustrations", "landscapes", "lighting effects", "low-poly", "materials", "model creation", "movement",
    "naturalism", "objects", "organic shapes", "painting", "photorealism", "pixelation", "polygonal", "portraits",
    "production", "realism", "renderings", "scenic design", "sculptural", "shaping", "simulations", "sketches",
    "software tools", "specularity", "structural design", "stylization", "surface textures", "technology", "texture mapping",
    "traditional art", "transformations", "visualization techniques", "volumetric lighting"
]

rpg_and_dnd_related_words = [
    "adventure", "alignment", "arcane", "armor", "bard", "battle", "campaign", "character", "cleric", "combat",
    "critical", "cult", "d20", "dungeon", "encounter", "epic", "fantasy", "fighter", "gamemaster", "guild", "hero",
    "magic", "monster", "mythical", "paladin", "quest", "rogue", "role-play", "sorcerer", "spell", "story", "tavern",
    "thief", "warlock", "wizard", "world", "ability", "artifact", "background", "beast", "challenge", "class",
    "creature", "darkness", "deity", "dragon", "encounter", "equipment", "experience", "faith", "fey", "goblin",
    "halfling", "history", "inn", "item", "legend", "lore", "mage", "mystery", "necromancer", "oracle", "orc",
    "power", "prophecy", "race", "relic", "scroll", "shaman", "sorcery", "spirit", "summon", "sword", "tale", "token",
    "trait", "trap", "troll", "villain", "weapon", "wilderness", "wonder", "adventure", "alignment", "arcane",
    "armor", "bard", "battle", "campaign", "character", "cleric", "combat", "critical", "cult", "d20", "dungeon",
    "encounter", "epic", "fantasy", "fighter", "gamemaster", "guild", "hero", "magic", "monster", "mythical",
    "paladin", "quest", "rogue", "role-play", "sorcerer", "spell", "story", "tavern", "thief", "warlock", "wizard",
    "world", "ability", "artifact", "background", "beast", "challenge", "class", "creature", "darkness", "deity",
    "dragon", "encounter", "equipment", "experience", "faith", "fey", "goblin", "halfling", "history", "inn", "item",
    "legend", "lore", "mage", "mystery", "necromancer", "oracle", "orc", "power", "prophecy", "race", "relic",
    "scroll", "shaman", "sorcery", "spirit", "summon", "sword", "tale", "token", "trait", "trap", "troll", "villain",
    "weapon", "wilderness", "wonder", "ability score", "adventure module", "alignment system", "arcane magic",
    "armor class", "bardic inspiration", "battle map", "campaign setting", "character sheet", "cleric domain",
    "combat encounter", "critical hit", "cultist ritual", "d20 system", "dungeon master", "encounter table",
    "epic fantasy", "fighter archetype", "gamemaster screen", "guildhall", "heroic deed", "magic item", "monster manual",
    "mythical creature", "paladin oath", "quest giver", "rogue archetype", "role-playing game", "sorcerer bloodline",
    "spellcaster", "story arc", "tavern brawl", "thief skill", "warlock patron", "wizard spellbook", "worldbuilding",
    "ability check", "artifact quest", "background story", "beastiary entry", "challenge rating", "character creation",
    "creature stat block", "dark fantasy", "deity worship", "dragon hoard", "encounter balance", "equipment shop",
    "experience points", "faithful follower", "feywild portal", "goblin horde", "halfling village", "history lesson",
    "innkeeper's secret", "item identification", "legendary hero", "lore master", "mage's tower", "mystery cult",
    "necromantic ritual", "oracle's prophecy", "orc warband", "power surge", "prophecy fulfilled", "racial trait",
    "relic hunt", "scroll deciphering", "shamanistic ritual", "sorcerer's spell", "spirit guardian", "summoned creature",
    "swordsmanship training", "tale retelling", "token of gratitude", "trait selection", "trap detection", "troll lair",
    "villainous plot", "weapon crafting", "wilderness survival", "wonderous item"
]

unity_game_dev_engineering_related_words = [
    "Unity", "game", "development", "engine", "software", "programming", "coding", "scripting", "C#", "CSharp",
    "programming language", "script", "codebase", "asset", "3D", "modeling", "animation", "physics", "rendering",
    "shaders", "lighting", "texturing", "terrain", "AI", "artificial intelligence", "pathfinding", "navigation",
    "user interface", "UI", "user experience", "UX", "input", "controls", "interaction", "multiplayer", "networking",
    "server", "client", "database", "backend", "frontend", "optimization", "performance", "debugging", "testing",
    "version control", "git", "repository", "collaboration", "integration", "build", "deployment", "platform",
    "web", "mobile", "desktop", "AR", "augmented reality", "VR", "virtual reality", "XR", "extended reality",
    "gameplay", "mechanics", "systems", "level design", "prototyping", "iteration", "feedback", "community",
    "documentation", "tutorials", "support", "forum", "community", "assets store", "plugin", "package", "SDK",
    "software development kit", "API", "application programming interface", "plugin", "extension", "tool", "utility",
    "framework", "library", "component", "prefab", "game object", "scene", "build", "player", "input manager",
    "audio manager", "animation controller", "particle system", "UI canvas", "scriptable object", "coroutine",
    "dependency injection", "singleton", "factory", "observer pattern", "event system", "state machine",
    "singleton pattern", "factory pattern", "observer pattern", "event-driven programming", "state machine",
    "object-oriented programming", "OOP", "data structure", "algorithm", "inheritance", "encapsulation", "polymorphism",
    "abstraction", "design pattern", "architectural pattern", "MVC", "model-view-controller", "MVVM", "model-view-viewmodel",
    "dependency injection", "composition over inheritance", "factory pattern", "singleton pattern", "object pool",
    "command pattern", "strategy pattern", "observer pattern", "flyweight pattern", "mediator pattern", "prototype pattern",
    "visitor pattern", "behavior tree", "finite state machine", "entity-component-system", "ECS", "data-oriented design",
    "performance optimization", "rendering pipeline", "forward rendering", "deferred rendering", "lightmapping",
    "real-time GI", "global illumination", "post-processing", "rendering optimization", "GPU", "CPU", "multi-threading",
    "job system", "burst compiler", "scripting runtime", "IL2CPP", "Mono", "backend architecture", "microservices",
    "serverless computing", "cloud computing", "DevOps", "continuous integration", "continuous deployment", "containerization",
    "Docker", "Kubernetes", "cloud platform", "AWS", "Amazon Web Services", "Azure", "Google Cloud Platform", "Firebase",
    "authentication", "authorization", "OAuth", "OpenID Connect", "security", "encryption", "SSL/TLS", "HTTPS"
]


In [96]:
def unique_items(list):
  unique_items = []
  for item in list:
        if item.lower() not in unique_items:
            unique_items.append(item.lower())

  return unique_items

game_dev_words = unique_items(game_dev_words)
hire_game_dev_words = unique_items(hire_game_dev_words)
vfx_related_words = unique_items(vfx_related_words)
digital_art_related_words = unique_items(digital_art_related_words)
marketing_related_words = unique_items(marketing_related_words)
modeling_and_sculpting_related_words = unique_items(modeling_and_sculpting_related_words)
rpg_and_dnd_related_words = unique_items(rpg_and_dnd_related_words)
unity_game_dev_engineering_related_words = unique_items(unity_game_dev_engineering_related_words)


In [97]:
"""
def list_to_csv (list, output_name):
    # Specify the file path where you want to save the CSV file
    csv_file_path = f"{output_name}.csv"
    # Open the CSV file in write mode
    with open(csv_file_path, mode='w', newline='') as file:
        # Create a CSV writer object
        writer = csv.writer(file)
        
        # Write each item in the final_list as a row in the CSV file
        for item in list:
            writer.writerow([item])

list_to_csv(vfx_related_words,'vfx_related_words')
list_to_csv(digital_art_related_words,'digital_art_related_words')
list_to_csv(marketing_related_words,'marketing_related_words')
list_to_csv(modeling_and_sculpting_related_words,'modeling_and_sculpting_related_words')
list_to_csv(hire_game_dev_words,'hire_game_dev_words')
list_to_csv(rpg_and_dnd_related_words,'rpg_and_dnd_related_words')
list_to_csv(unity_game_dev_engineering_related_words,'unity_game_dev_engineering_related_words')
"""

'\ndef list_to_csv (list, output_name):\n    # Specify the file path where you want to save the CSV file\n    csv_file_path = f"{output_name}.csv"\n    # Open the CSV file in write mode\n    with open(csv_file_path, mode=\'w\', newline=\'\') as file:\n        # Create a CSV writer object\n        writer = csv.writer(file)\n        \n        # Write each item in the final_list as a row in the CSV file\n        for item in list:\n            writer.writerow([item])\n\nlist_to_csv(vfx_related_words,\'vfx_related_words\')\nlist_to_csv(digital_art_related_words,\'digital_art_related_words\')\nlist_to_csv(marketing_related_words,\'marketing_related_words\')\nlist_to_csv(modeling_and_sculpting_related_words,\'modeling_and_sculpting_related_words\')\nlist_to_csv(hire_game_dev_words,\'hire_game_dev_words\')\nlist_to_csv(rpg_and_dnd_related_words,\'rpg_and_dnd_related_words\')\nlist_to_csv(unity_game_dev_engineering_related_words,\'unity_game_dev_engineering_related_words\')\n'

In [98]:
general_hiring_worksheet = sheet.worksheet('General Hiring')
RPG_DnD_worksheet = sheet.worksheet('RPG, D&D')
Marketing_worksheet = sheet.worksheet('Marketing')
twoD_worksheet = sheet.worksheet('2D')
threeD_worksheet = sheet.worksheet('3D')
VFX_hiring_worksheet = sheet.worksheet('VFX')
unity_Engineering_worksheet = sheet.worksheet('Unity Engineering')


general_hiring_title = general_hiring_worksheet.title
RPG_DnD_title = RPG_DnD_worksheet.title
Marketing_title = Marketing_worksheet.title
twoD_worksheet_title = twoD_worksheet.title
threeD_worksheet_title = threeD_worksheet.title
VFX_hiring_worksheet_title = VFX_hiring_worksheet.title
unity_Engineering_title = unity_Engineering_worksheet.title



def get_value_list_from_sheet(sheet):
    values_list = sheet.col_values(1)
    return values_list

general_hiring = get_value_list_from_sheet(general_hiring_worksheet)
RPG_DnD = get_value_list_from_sheet(RPG_DnD_worksheet)
Marketing = get_value_list_from_sheet(Marketing_worksheet)
twoD = get_value_list_from_sheet(twoD_worksheet)
threeD = get_value_list_from_sheet(threeD_worksheet)
VFX_hiring = get_value_list_from_sheet(VFX_hiring_worksheet)
unity_Engineering = get_value_list_from_sheet(unity_Engineering_worksheet)

In [99]:
import json

# Chemin vers le fichier JSON
json_file_path = 'credentials.json'

# Lire les données du fichier JSON
with open(json_file_path, 'r') as json_file:
    credentials = json.load(json_file)

reddit = praw.Reddit(client_id=credentials['client_id'],
                     client_secret=credentials['client_secret'],
                     user_agent=credentials['user_agent'])

In [100]:
# Définition des paramètres pour la recherche de mots-clés
num_keywords = 4          # Nombre de mots-clés aléatoires à sélectionner pour chaque recherche
post_limits = 150         # Nombre maximal de publications à récupérer par recherche
iterations = 30           # Nombre d'itérations (c'est-à-dire nombre de recherches)
keywords_list = unity_Engineering
# Définition des colonnes du DataFrame
columns = ["title", "subreddit", "score", "url", "created_utc", "num_comments", "content"]


results = pd.DataFrame(columns=columns)

# Fonction pour rechercher des publications Reddit avec des mots-clés aléatoires
def search_reddit_with_keywords(list, num_keywords, limit):
    global results  # Declare results as a global variable
    # Sélectionner des mots-clés aléatoires à partir de la liste fournie
    random_keywords = random.choices(list, k=num_keywords)
    print(f"Recherche avec les mots-clés aléatoires : {random_keywords}")

    # Construire une chaîne de requête de recherche avec une logique ET pour tous les mots-clés sélectionnés
    search_query = ' AND '.join(random_keywords)
    
    # Performer la recherche sur Reddit
    for submission in reddit.subreddit('all').search(search_query, limit=limit):
        # Vérifier si la publication n'existe pas déjà dans le DataFrame en fonction de son URL
        if submission.url not in results['url'].values:
            # Formatter le timestamp created_utc au format JJ/MM/AAAA
            created_date = datetime.datetime.fromtimestamp(submission.created_utc, datetime.timezone.utc).strftime('%d/%m/%Y')
            
            # Créer un dictionnaire temporaire avec le nouveau résultat de recherche
            temp_dict = {
                "title": str(submission.title),
                "subreddit": str(submission.subreddit),
                "score": int(submission.score),
                "url": str(submission.url),
                "created_utc": created_date,
                "num_comments": int(submission.num_comments),
                "content": str(submission.selftext)
            }
            
            results = pd.concat([results, pd.DataFrame([temp_dict])], ignore_index=True)

    return results

# Boucle principale pour effectuer les recherches itératives
for i in range(iterations):
    print(f"Itération {i+1}")
    
    # Effectuer la recherche Reddit avec les mots-clés
    results = search_reddit_with_keywords(keywords_list, num_keywords=num_keywords, limit=post_limits)
    
    # Supprimer les doublons basés sur l'URL pour assurer l'unicité
    results.drop_duplicates(subset=['url'], keep='first', inplace=True)
    
    # Imprimer le nombre de lignes dans les résultats
    print(f"Nombre de lignes dans les résultats : {results.shape[0]}")
    
    # Attendre avant la prochaine itération
    time.sleep(3)

print(f"Les résultats ont été enregistrés  into the DataFrame")

# Fusionner le contenu des colonnes 'title' et 'content' et le mettre dans la colonne 'content'
results['content'] = results.apply(lambda row: row['title'] + ': ' + row['content'], axis=1)
results = results.drop(columns='title')
# Convertir les colonnes 'score' et 'num_comments' en types entiers
results['score'] = pd.to_numeric(results['score'], errors='coerce').astype('Int64')
results['num_comments'] = pd.to_numeric(results['num_comments'], errors='coerce').astype('Int64')

# Convertir la colonne 'created_utc' en type datetime
results['created_utc'] = pd.to_datetime(results['created_utc'], format='%d/%m/%Y', errors='coerce')


# Function to remove URLs from text
def remove_links(text):
    if isinstance(text, str):  # Check if the input is a string
        url_pattern = r'https?://\S+|www\.\S+'
        cleaned_text = re.sub(url_pattern, '', text)
        return cleaned_text
    return text  # If not a string (e.g., NaN), return as is

# Function to clean text
def clean_text(text):
    if isinstance(text, str):  # Check if the input is a string
        # Remove zero-width space characters
        text = text.replace('\u200b', '').replace('\x200b', '')
        # Expand contractions
        text = contractions.fix(text)
        # Remove non-alphanumeric characters (except spaces)
        text = re.sub(r'\W+', ' ', text)
        return text.strip()
    return text  # If not a string (e.g., NaN), return as is

# Function to remove specific unwanted words
def remove_specific_words(text):
    if isinstance(text, str):  # Check if the input is a string
        # Define words to be removed
        words_to_remove = ['x200b', 'u200b', 'pdf', 'ebook']
        for word in words_to_remove:
            text = text.replace(word, '')
        return text.strip()
    return text  # If not a string (e.g., NaN), return as is

# Apply the cleaning functions to the 'content' column
results['content'] = results['content'].apply(remove_links).apply(clean_text).apply(remove_specific_words)


# Create a boolean mask for rows containing URLs
mask = results['content'].str.contains(r'https?://|www\.|\b\.com\b', case=False)
results = results.dropna(subset = ['content'])
if keywords_list == general_hiring:
    csv_file = f'{general_hiring_title}.csv'
elif keywords_list == RPG_DnD:
    csv_file = f'{RPG_DnD_title}.csv'
elif keywords_list == Marketing:
    csv_file = f'{Marketing_title}.csv'
elif keywords_list == twoD:
    csv_file = f'{twoD_worksheet_title}.csv'
elif keywords_list == threeD:
    csv_file = f'{threeD_worksheet_title}.csv'
elif keywords_list == VFX_hiring:
    csv_file = f'{VFX_hiring_worksheet_title}.csv'
elif keywords_list == unity_Engineering:
    csv_file = f'{unity_Engineering_title}.csv'

# Check if the CSV file exists
if not os.path.isfile(csv_file):
    # If the file doesn't exist, create a new CSV file with the DataFrame
    results.to_csv(csv_file, index=False)
    print(f"CSV file '{csv_file}' created successfully.")
else:
    # If the file already exists, append the DataFrame to the existing CSV file
    results.to_csv(csv_file, mode='a', header=False, index=False)
    print(f"Data appended to the existing CSV file '{csv_file}'.")



Itération 1
Recherche avec les mots-clés aléatoires : ['behavior tree', 'https', 'performance optimization', 'ar']
Nombre de lignes dans les résultats : 40
Itération 2
Recherche avec les mots-clés aléatoires : ['api', 'sdk', 'strategy pattern', 'codebase']
Nombre de lignes dans les résultats : 41
Itération 3
Recherche avec les mots-clés aléatoires : ['containerization', 'feedback', 'xr', 'global illumination']
Nombre de lignes dans les résultats : 41
Itération 4
Recherche avec les mots-clés aléatoires : ['codebase', 'state machine', 'prototyping', 'ai']
Nombre de lignes dans les résultats : 52
Itération 5
Recherche avec les mots-clés aléatoires : ['openid connect', 'event system', 'cloud platform', 'cloud platform']
Nombre de lignes dans les résultats : 56
Itération 6
Recherche avec les mots-clés aléatoires : ['testing', 'ui canvas', 'mvvm', 'data structure']
Nombre de lignes dans les résultats : 57
Itération 7
Recherche avec les mots-clés aléatoires : ['extension', 'codebase', 'job sy

In [ ]:
results.head()

In [103]:
# Define the vectorizer with desired parameters
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

# Convert text data into document-term matrix
X = vectorizer.fit_transform(results['content'])

# Initialize and fit the LDA model
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(X)

# Function to get main topic for each content row along with main words
def get_main_topic_and_words(model, vectorizer, text, n_words=10):
    # Transform text into document-term matrix
    text_matrix = vectorizer.transform([text])
    # Infer topic distribution for the text
    topic_distribution = model.transform(text_matrix)
    # Get main topic index
    main_topic_index = topic_distribution.argmax()
    # Get main topic words
    feature_names = vectorizer.get_feature_names_out()
    main_topic_words_indices = model.components_[main_topic_index].argsort()[:-n_words - 1:-1]
    main_topic_words = [feature_names[i] for i in main_topic_words_indices]
    return main_topic_index, main_topic_words

# Add 'Main Topic' and 'Main Words' columns to 'results' DataFrame
results['Main Topic'], results['Main Words'] = zip(*results['content'].apply(
    lambda x: get_main_topic_and_words(lda_model, vectorizer, x)))

In [ ]:
results.head()

In [104]:
grouped_results = results.groupby('Main Topic').agg({
    'Main Topic': 'count',  # Count the occurrences of each main topic
    'Main Words': lambda x: ', '.join(set(word for sublist in x for word in sublist))  # Join words into a single string, then convert to set to get unique words, then join
})

grouped_results.head()

,Main Topic,Main Words
Main Topic,,
0,158,"damage, fixed, added, use, players, new, map, issue, game, player"
1,84,"brain, people, pdf, time, new, ebook, edition, human, world, like"
2,136,"make, time, design, just, work, ai, team, game, development, like"
3,224,"experience, company, gamestop, use, user, data, software, network, support, systems"
4,138,"said, did, going, know, way, time, ship, just, x200b, like"


In [ ]:
# Perform the aggregation
grouped_by_subreddit = results.groupby('subreddit').agg({
    'num_comments': 'sum',     # Sum of comments for each subreddit
    'score': 'mean',           # Mean score for each subreddit
    'created_utc': 'count'     # Count of posts for each subreddit
})

# Display the aggregated DataFrame
grouped_by_subreddit.head(5)


,num_comments,score,created_utc
subreddit,,,
2D3DAI,0,4.5,2
3DKanojo,0,8.0,1
3Dmodeling,20,21.0,1
3Dprinting,0,1.0,1
3danimation,1,1.0,1


ML Preparation